# NOT WORKING

In [5]:
from bitsandbytes.optim import Adam8bit


In [4]:

from accelerate import Accelerator
accelerator = Accelerator(fp16=1)


TypeError: Accelerator.__init__() got an unexpected keyword argument 'fp16'

In [1]:
from transformers import GPT2LMHeadModel, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('gpt2')


In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    output_dir='/home/arjun/Desktop/test'
)

In [ ]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer,GPT2LMHeadModel, get_scheduler 
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
dataset = load_dataset("tatsu-lab/alpaca")
dataset = dataset['train']
tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
dataset = dataset.select(range(100))
new_dataset = {'input_ids':[], 'attention_mask':[]}

for example in dataset:
    input_text = example['text'].replace('###','')   
    encoded_data = tokenizer('<|startoftext|>' + input_text + '<|endoftext|>',truncation=True, max_length=768, padding="max_length")
    new_dataset['input_ids'].append(encoded_data['input_ids'])
    new_dataset['attention_mask'].append(encoded_data['attention_mask'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")
dataloader = DataLoader(new_dataset,shuffle=True, batch_size=8)

In [ ]:
num_epochs=2
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [ ]:
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

# dataloader = DataLoader(ds, batch_size=training_args.per_device_train_batch_size)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator()
model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)



In [ ]:
model.train()
for step, batch in enumerate(dataloader, start=1):
    loss = model(**batch).loss
    loss = loss / training_args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % training_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
from transformers import GPT2LMHeadModel, AutoTokenizer, get_scheduler 
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from datasets import load_dataset, Dataset
from accelerate import Accelerator


model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')


In [ ]:

dataset = load_dataset("tatsu-lab/alpaca")
dataset = dataset['train']
dataset = dataset.select(range(100))
new_dataset = {'input_ids':[], 'attention_mask':[]}

for example in dataset:
    input_text = example['text'].replace('###','')   
    encoded_data = tokenizer('' + input_text + '', truncation=True, max_length=768, padding="max_length")
    new_dataset['input_ids'].append(encoded_data['input_ids'])
    new_dataset['attention_mask'].append(encoded_data['attention_mask'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")
dataloader = DataLoader(new_dataset, shuffle=True, batch_size=8)


In [2]:
from transformers import TrainingArguments

num_epochs = 2
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    output_dir='/home/arjun/Desktop/test'
)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator()
model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)

model.train()
for step, batch in enumerate(dataloader, start=1):
    loss = model(**batch).loss
    loss = loss / training_args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % training_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize`

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`